In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import exdir
import exdir.plugins.quantities
import exdir.plugins.git_lfs
import seaborn as sns
import os
import expipe
import pathlib
import numpy as np
import place_stimulation as ps
import neo
import quantities as pq
import matplotlib.pylab as plt

In [3]:
project_path = r'C:\Users\dejanam\data\test_pcstim'
project = expipe.get_project(project_path)
actions = project.actions
actions

In [4]:
action = project.actions['1849-260219-2']
data_path = ps.get_data_path(action)

C:\Users\dejanam\data\test_pcstim
actions\1849-260219-2\data\main.exdir


In [ ]:
sptr = ps.load_spiketrains(data_path, channel_idx=6, remove_label='noise')
st = sptr[0]
print(st.waveforms.shape)

(14463, 4, 90)


In [ ]:
import matplotlib.pylab as plt
import spatialspikefields as tr
from scipy.interpolate import interp1d
import quantities as pq
import numpy as np
import seaborn as sns
import matplotlib.gridspec as gridspec


def plot_all_recordings(sptr,  data_path, color='b', lw=2, par=None, s=30, c=[0.7, 0.2, 0.2],
                        select_tracking=None, fig=None, figsize=(20, 20),
                        binsize = 0.02, smoothing = 0.03):
    fig = plt.figure(constrained_layout=True, figsize=figsize)
    sns.set(color_codes=True, style="darkgrid")
    gs = gridspec.GridSpec(1, 3, figure=fig)

    # creating axes for waveforms
    gs_wf = gridspec.GridSpecFromSubplotSpec(4, 1, subplot_spec=gs[0])
    #gs_wf.update(left=0.05, right=0.48)
    axs_wf = []
    for i in range(4):
        ax_wf = fig.add_subplot(gs_wf[i, :])
        axs_wf.append(ax_wf)

    for i in range(4):
        wf = sptr[0].waveforms[:, i, :]
        m = np.mean(wf, axis=0)
        s_time = np.arange(m.size, dtype=np.float32) / sptr[0].sampling_rate
        s_time.units = 'ms'
        sd = np.std(wf, axis=0)
        axs_wf[i].plot(s_time, m, color=color, lw=lw)
        axs_wf[i].fill_between(s_time, m - sd, m + sd, alpha=.1, color=color)
        #if sptr.left_sweep is not None:
         #   sptr.left_sweep.units = 'ms'
          #  axs[c].axvspan(sptr.left_sweep, sptr.left_sweep, color='k',
           #                ls='--')
        axs_wf[i].set_xlabel(s_time.dimensionality)
        axs_wf[i].set_xlim([s_time.min(), s_time.max()])
        plt.setp(axs_wf[i].get_yticklabels(), visible=False)
        if i < 3:
            plt.setp(axs_wf[i].get_xticklabels(), visible=False)
            
    # plotting path
    if par is None:
        par = {'speed_filter': 0.3,  # m/s
               'pos_fs': 60,
               'f_cut': 1,
               'spat_binsize': 0.02,
               'spat_smoothing': 0.025,
               'grid_stepsize': 0.1,
               'box_xlen': 1,
               'box_ylen': 1,
               'spike_size': 10,
               'field_max_wall_extent': 0.1,
               'field_min_bins': 12
               }

    # creating axes for path
    x, y, t, speed = ps.load_tracking(data_path, par, select_tracking=1)
    gs_p = gridspec.GridSpecFromSubplotSpec(4, 1, subplot_spec=gs[1])
    axs_p = []
    for i in range(4):
        axs_p.append(plt.subplot(gs_p[i, :]))
        axs_p[i].plot(x, y, 'k', alpha=0.3)

    x_spikes = []
    y_spikes = []
    for i in range(4):
        sptr_t = sptr[0][sptr[0].times.magnitude < np.max(t)]
        sptr_t = sptr_t[sptr_t.times.magnitude > np.min(t)]
        x_spikes.append(interp1d(t, x)(sptr_t))
        y_spikes.append(interp1d(t, y)(sptr_t))

    for ax, i in zip(axs_p, range(4)):
        ax.scatter(x_spikes[i], y_spikes[i], s=s, c=c)

    plt.xticks([])
    plt.yticks([])

    # plotting firing-rate maps
    gs_fr = gridspec.GridSpecFromSubplotSpec(4, 1, subplot_spec=gs[2])
    axs_fr = []
    for i in range(4):
        axs_fr.append(plt.subplot(gs_fr[i, :]))
        rate_map = tr.spatial_rate_map(x, y, t, sptr[0], binsize=binsize, box_xlen=1, box_ylen=1, smoothing=smoothing)
        axs_fr[i].imshow(rate_map, origin="lower")

    plt.xticks([])
    plt.yticks([])

    plt.savefig("all-in-one-plot.png")

    return plt.show()

In [ ]:
plot_all_recordings(sptr, data_path, color='b', lw=2, par=None, s=30, c=[0.7, 0.2, 0.2],
                        select_tracking=None, fig=None, figsize=(20, 20),
                        binsize = 0.02, smoothing = 0.03)